In [1]:
import yaml
import os
import requests

In [2]:
def load_secrets():
    '''Load Spotify Client ID and Client Secret from "~/.secrets/spotify_api.yaml" '''
    secret_file = os.path.join(os.environ['HOME'], '.secrets/spotify_api.yaml')
    with open(secret_file) as f:
        secrets = yaml.load(f)
    return secrets

In [3]:
def get_token(secrets):
    """Get an access token for the Spotify API."""
    url = "https://accounts.spotify.com/api/token"
    payload = secrets.copy()
    payload['grant_type'] = 'client_credentials'
    
    r = requests.post(url, data=payload)
    return r.json()

In [4]:
def spotify_request(url, token, params):
    """Make a Spotify request using the specified token and parameters."""
    r = requests.get(url, headers={'Authorization': 'Bearer {}'.format(token['access_token'])},
                     params=params)
    return r.json()


In [5]:
secrets = load_secrets()
token = get_token(secrets)

In [6]:
# # testing getting an album:

# url = "https://api.spotify.com/v1/albums/"
# # params = {"location": "111 S Jackson St, Seattle, WA 98104", "radius": 100}
# # businesses_near_galvanize = yelp_request(url, token, params)

# params ={id}
# get_tracks = spotify_request(url, token, params)

In [7]:
# get artist albums
r = requests.get("https://api.spotify.com/v1/albums/0sNOF9WDwhWunNAHPD3Baj", 
                 headers={'Authorization': 'Bearer {}'.format(token['access_token'])})
r.json()

{u'album_type': u'album',
 u'artists': [{u'external_urls': {u'spotify': u'https://open.spotify.com/artist/2BTZIqw0ntH9MvilQ3ewNY'},
   u'href': u'https://api.spotify.com/v1/artists/2BTZIqw0ntH9MvilQ3ewNY',
   u'id': u'2BTZIqw0ntH9MvilQ3ewNY',
   u'name': u'Cyndi Lauper',
   u'type': u'artist',
   u'uri': u'spotify:artist:2BTZIqw0ntH9MvilQ3ewNY'}],
 u'available_markets': [],
 u'copyrights': [{u'text': u'(P) 2000 Sony Music Entertainment Inc.',
   u'type': u'P'}],
 u'external_ids': {u'upc': u'5099749994324'},
 u'external_urls': {u'spotify': u'https://open.spotify.com/album/0sNOF9WDwhWunNAHPD3Baj'},
 u'genres': [],
 u'href': u'https://api.spotify.com/v1/albums/0sNOF9WDwhWunNAHPD3Baj',
 u'id': u'0sNOF9WDwhWunNAHPD3Baj',
 u'images': [{u'height': 640,
   u'url': u'https://i.scdn.co/image/b6e762dcce1502ce63eb2c68798843eb2ed53c51',
   u'width': 640},
  {u'height': 300,
   u'url': u'https://i.scdn.co/image/66302ca80395b6be3600d5f0ef69db9e0c43f4f5',
   u'width': 300},
  {u'height': 64,
   u'url'

In [30]:
# search for an item
r = requests.get("https://api.spotify.com/v1/search?q=tania%20bowra&type=artist", 
                 headers={'Authorization': 'Bearer {}'.format(token['access_token'])})
r.json()


{u'artists': {u'href': u'https://api.spotify.com/v1/search?query=tania+bowra&type=artist&offset=0&limit=20',
  u'items': [{u'external_urls': {u'spotify': u'https://open.spotify.com/artist/08td7MxkoHQkXnWAYD8d6Q'},
    u'followers': {u'href': None, u'total': 115},
    u'genres': [],
    u'href': u'https://api.spotify.com/v1/artists/08td7MxkoHQkXnWAYD8d6Q',
    u'id': u'08td7MxkoHQkXnWAYD8d6Q',
    u'images': [{u'height': 640,
      u'url': u'https://i.scdn.co/image/a529b65b4bd322b16bee34672ce45278e890e176',
      u'width': 640},
     {u'height': 300,
      u'url': u'https://i.scdn.co/image/985cc10acdbbedb6a16d7c74f9e23553e2b28dbc',
      u'width': 300},
     {u'height': 64,
      u'url': u'https://i.scdn.co/image/37b46a2662c09502885d1804c1c865b199cc3d67',
      u'width': 64}],
    u'name': u'Tania Bowra',
    u'popularity': 2,
    u'type': u'artist',
    u'uri': u'spotify:artist:08td7MxkoHQkXnWAYD8d6Q'}],
  u'limit': 20,
  u'next': None,
  u'offset': 0,
  u'previous': None,
  u'total': 

In [8]:
import time
import urllib
import webbrowser
from urlparse import urlparse


import requests
from requests.auth import HTTPBasicAuth

OAUTH_AUTHORIZE_URL = 'https://accounts.spotify.com/authorize'
OAUTH_TOKEN_URL = 'https://accounts.spotify.com/api/token'

# Change to your application settings
class Settings:
    client_id = secrets['client_id']
    client_secret = secrets['client_secret']
    redirect_uri = 'http://localhost:8080/callback'


def authenticate(scope=None):
    '''Implement OAuth 2 Spotify authentication'''
    # Application: Request authorization to access data
    payload = {'client_id': Settings.client_id,
               'response_type': 'code',
               'redirect_uri': Settings.redirect_uri,
               'show_dialog': 'false'} # allow second account to login
    if scope:
        payload['scope'] = scope
    auth_url = '{}?{}'.format(OAUTH_AUTHORIZE_URL, urllib.urlencode(payload))
    # Spotify: Displays scopes & prompts user to login (if required)
    # User: Logs in, authorizes access
    webbrowser.open(auth_url)

    response = input('Enter the URL you were redirected to: ')
    code = urllib.parse_qs(urllib.parse(response).query)['code'][0]

    payload = {'redirect_uri': Settings.redirect_uri,
               'code': code,
               'grant_type': 'authorization_code'}
    if scope:
        payload['scope'] = scope

    # Application: Request access and refresh tokens
    # Spotify: Returns access and refresh tokens
    auth = HTTPBasicAuth(Settings.client_id, Settings.client_secret)
    response = requests.post(OAUTH_TOKEN_URL, data=payload, auth=auth)
    if response.status_code != 200:
        response.raise_for_status()
    token_info = response.json()
    token_info['expires_at'] = int(time.time()) + token_info['expires_in']
    token_info['scope'] = scope
    return token_info



In [9]:
user_a = authenticate(scope='user-library-read')


KeyboardInterrupt: 

In [ ]:
import json
from flask import Flask, request, redirect, g, render_template
import requests
import base64
import urllib

# Authentication Steps, paramaters, and responses are defined at https://developer.spotify.com/web-api/authorization-guide/
# Visit this url to see all the steps, parameters, and expected response. 


app = Flask(__name__)

#  Client Keys
CLIENT_ID = secrets['client_id']
CLIENT_SECRET = secrets['client_secret']
# redirect_uri = 'http://localhost:8080/callback'

# Spotify URLS
SPOTIFY_AUTH_URL = "https://accounts.spotify.com/authorize"
SPOTIFY_TOKEN_URL = "https://accounts.spotify.com/api/token"
SPOTIFY_API_BASE_URL = "https://api.spotify.com"
API_VERSION = "v1"
SPOTIFY_API_URL = "{}/{}".format(SPOTIFY_API_BASE_URL, API_VERSION)


# Server-side Parameters
CLIENT_SIDE_URL = "http://localhost"
PORT = 8080
REDIRECT_URI = "{}:{}/callback".format(CLIENT_SIDE_URL, PORT)
SCOPE = "user-library-read"
STATE = ""
SHOW_DIALOG_bool = True
SHOW_DIALOG_str = str(SHOW_DIALOG_bool).lower()


auth_query_parameters = {
    "response_type": "code",
    "redirect_uri": REDIRECT_URI,
    "scope": SCOPE,
    # "state": STATE,
    # "show_dialog": SHOW_DIALOG_str,
    "client_id": CLIENT_ID
}

@app.route("/")
def index():
    # Auth Step 1: Authorization
    url_args = "&".join(["{}={}".format(key,urllib.quote(val)) for key,val in auth_query_parameters.iteritems()])
    auth_url = "{}/?{}".format(SPOTIFY_AUTH_URL, url_args)
    return redirect(auth_url)


@app.route("/callback")
def callback():
    # Auth Step 4: Requests refresh and access tokens
    auth_token = request.args['code']
    code_payload = {
        "grant_type": "authorization_code",
        "code": str(auth_token),
        "redirect_uri": REDIRECT_URI
    }
    base64encoded = base64.b64encode("{}:{}".format(CLIENT_ID, CLIENT_SECRET))
    headers = {"Authorization": "Basic {}".format(base64encoded)}
    post_request = requests.post(SPOTIFY_TOKEN_URL, data=code_payload, headers=headers)

    # Auth Step 5: Tokens are Returned to Application
    response_data = json.loads(post_request.text)
    access_token = response_data["access_token"]
    refresh_token = response_data["refresh_token"]
    token_type = response_data["token_type"]
    expires_in = response_data["expires_in"]

    # Auth Step 6: Use the access token to access Spotify API
    authorization_header = {"Authorization":"Bearer {}".format(access_token)}

    # Get profile data
    user_profile_api_endpoint = "{}/me".format(SPOTIFY_API_URL)
    profile_response = requests.get(user_profile_api_endpoint, headers=authorization_header)
    profile_data = json.loads(profile_response.text)

    # Get user playlist data
    playlist_api_endpoint = "{}/playlists".format(profile_data["href"])
    playlists_response = requests.get(playlist_api_endpoint, headers=authorization_header)
    playlist_data = json.loads(playlists_response.text)
    
    # Combine profile and playlist data to display
    display_arr = [profile_data] + playlist_data["items"]
    return render_template("index.html",sorted_array=display_arr)


In [ ]:
app.run(debug=True,port=PORT)